In [ ]:
### MultiNicheNetR Analysis on EOAD
## Focusing on specific cell types between EOAD and control 
# Andi Liu

# loading
library(Seurat)
library(Signac)
library(dplyr)
library(patchwork)
library(ggplot2)
library(future)
library(stringr)
library(tidydr)
library(RColorBrewer)
library(cowplot)
library(ggpubr)
library(tidyverse)

library(SingleCellExperiment)
library(dplyr)
library(ggplot2)
library(multinichenetr)

# set working dir
setwd("/data2/aliu8/2023_AD_multiome/Analysis/")

In [ ]:
### loding objects
gc()
#### process the results
pfc <- readRDS("cellbender_PFC_object.rds")
hip <- readRDS("cellbender_HIP_object.rds")
ec <- readRDS("cellbender_EC_object.rds")

pfc
ec
hip
ls()

In [ ]:
organism = "human"
if(organism == "human"){
  lr_network = readRDS(url("https://zenodo.org/record/7074291/files/lr_network_human_21122021.rds"))
  lr_network = lr_network %>% dplyr::rename(ligand = from, receptor = to) %>% distinct(ligand, receptor) %>% mutate(ligand = make.names(ligand), receptor = make.names(receptor))
  ligand_target_matrix = readRDS(url("https://zenodo.org/record/7074291/files/ligand_target_matrix_nsga2r_final.rds"))
  colnames(ligand_target_matrix) = colnames(ligand_target_matrix) %>% make.names()
  rownames(ligand_target_matrix) = rownames(ligand_target_matrix) %>% make.names()
} else if(organism == "mouse"){
  lr_network = readRDS(url("https://zenodo.org/record/7074291/files/lr_network_mouse_21122021.rds"))
  lr_network = lr_network %>% dplyr::rename(ligand = from, receptor = to) %>% distinct(ligand, receptor) %>% mutate(ligand = make.names(ligand), receptor = make.names(receptor))
  ligand_target_matrix = readRDS(url("https://zenodo.org/record/7074291/files/ligand_target_matrix_nsga2r_final_mouse.rds"))
  colnames(ligand_target_matrix) = colnames(ligand_target_matrix) %>% make.names()
  rownames(ligand_target_matrix) = rownames(ligand_target_matrix) %>% make.names()
}

In [ ]:
Idents(pfc) <- "cluster_celltype"
Idents(ec) <- "cluster_celltype"
Idents(hip) <- "cluster_celltype"

In [ ]:
## WORKING on each region
## keeping selected celltypes
obj <- subset(x = ec, idents = c("Astrocyte", "Excitatory","Inhibitory","Microglia","Oligodendrocyte","OPC"))
obj$cluster_celltype <- as.character(obj$cluster_celltype)

In [ ]:
## convert seurat object into sec object
sce = Seurat::as.SingleCellExperiment(obj, assay = "PC")
sce = alias_to_symbol_SCE(sce, "human") %>% makenames_SCE()

## setting variates
sample_id = "individual_ID"
group_id = "diagnosis"
celltype_id = "cluster_celltype"
covariates = NA
batches = NA

## filter the cell type of interest
senders_oi = SummarizedExperiment::colData(sce)[,celltype_id] %>% unique()
receivers_oi = SummarizedExperiment::colData(sce)[,celltype_id] %>% unique()
## excluding rare cell type in the object
sce = sce[, SummarizedExperiment::colData(sce)[,celltype_id] %in% c(senders_oi, receivers_oi)]
sce

In [ ]:
### quality control procedures for the selected region

## define gorup
min_cells = 10

## check abundance of cells 
abundance_expression_info <-  get_abundance_expression_info(
    sce = sce,
    sample_id = sample_id, 
    group_id = group_id, 
    celltype_id = celltype_id, 
    min_cells = min_cells, 
    senders_oi = senders_oi, 
    receivers_oi = receivers_oi, 
    lr_network = lr_network, 
    batches = batches)

# set case anc control
contrasts_oi = c("'EOAD-NCI','NCI-EOAD'")
contrast_tbl = tibble(contrast =
                        c("EOAD-NCI", "NCI-EOAD"),
                      group = c("EOAD", "NCI"))

        
## performing DE basded on parameters
DE_info <- get_DE_info(sce = sce, 
                        sample_id = sample_id, 
                        group_id = group_id, 
                        celltype_id = celltype_id, 
                        batches = batches, 
                        covariates = covariates, 
                        contrasts_oi = contrasts_oi, 
                        min_cells = min_cells)

## check how many DEGs calculated by used method
#DE_info$celltype_de$de_output_tidy %>% arrange(-p_val) %>% head()
de_check <- DE_info$celltype_de$de_output_tidy
table(de_check[de_check$p_val <= 0.05,]$cluster_id)

deg <- read.csv("./Results/DEG/Overlap_mast_mixed_EC.csv",row.names = 1)
head(deg)

df <- DE_info$celltype_de$de_output_tidy
df$dir <- ifelse(df$logFC >0, "pos","neg")
df$comb <- paste(df$gene, df$dir, df$cluster_id, sep = "_")
DE_info$celltype_de$de_output_tidy <- DE_info$celltype_de$de_output_tidy[df$comb %in% deg$comb,]
#DE_info$celltype_de$de_output_tidy

In [ ]:
## using default parameters
# using nominal p values because of small sample size
logFC_threshold = 0.75
p_val_threshold = 0.05
fraction_cutoff = 0.25

# if using adjusted p value
p_val_adj = FALSE

top_n_target = 250

verbose = TRUE
cores_system = 12
n.cores = min(cores_system, sender_receiver_de$receiver %>% unique() %>% length())

ligand_activities_targets_DEgenes = suppressMessages(suppressWarnings(get_ligand_activities_targets_DEgenes(
  receiver_de = celltype_de,
  receivers_oi = receivers_oi,
  ligand_target_matrix = ligand_target_matrix,
  logFC_threshold = logFC_threshold,
  p_val_threshold = p_val_threshold,
  p_val_adj = p_val_adj,
  top_n_target = top_n_target,
  verbose = verbose, 
  n.cores = n.cores
)))

## using default parameters
prioritizing_weights_DE = c("de_ligand" = 1,
                         "de_receptor" = 1)
prioritizing_weights_activity = c("activity_scaled" = 2)

prioritizing_weights_expression_specificity = c("exprs_ligand" = 2,
                         "exprs_receptor" = 2)

prioritizing_weights_expression_sufficiency = c("frac_exprs_ligand_receptor" = 1)

prioritizing_weights_relative_abundance = c( "abund_sender" = 0,
                         "abund_receiver" = 0)

                         

In [ ]:
prioritizing_weights = c(prioritizing_weights_DE, 
                         prioritizing_weights_activity, 
                         prioritizing_weights_expression_specificity,
                         prioritizing_weights_expression_sufficiency, 
                         prioritizing_weights_relative_abundance)

sender_receiver_tbl = sender_receiver_de %>% dplyr::distinct(sender, receiver)

metadata_combined = SummarizedExperiment::colData(sce) %>% tibble::as_tibble()

if(!is.na(batches)){
  grouping_tbl = metadata_combined[,c(sample_id, group_id, batches)] %>% tibble::as_tibble() %>% dplyr::distinct()
  colnames(grouping_tbl) = c("sample","group",batches)
} else {
  grouping_tbl = metadata_combined[,c(sample_id, group_id)] %>% tibble::as_tibble() %>% dplyr::distinct()
  colnames(grouping_tbl) = c("sample","group")
}

prioritization_tables = suppressMessages(generate_prioritization_tables(
  sender_receiver_info = abundance_expression_info$sender_receiver_info,
  sender_receiver_de = sender_receiver_de,
  ligand_activities_targets_DEgenes = ligand_activities_targets_DEgenes,
  contrast_tbl = contrast_tbl,
  sender_receiver_tbl = sender_receiver_tbl,
  grouping_tbl = grouping_tbl,
  prioritizing_weights = prioritizing_weights,
  fraction_cutoff = fraction_cutoff, 
  abundance_data_receiver = abundance_expression_info$abundance_data_receiver,
  abundance_data_sender = abundance_expression_info$abundance_data_sender

## Add information on prior knowledge and expression correlation between LR and target expression
lr_target_prior_cor <- lr_target_prior_cor_inference(
    prioritization_tables$group_prioritization_tbl$receiver %>% unique(),
    abundance_expression_info, 
    celltype_de, 
    grouping_tbl, 
    prioritization_tables, 
    ligand_target_matrix, 
    logFC_threshold = logFC_threshold, 
    p_val_threshold = p_val_threshold, 
    p_val_adj = p_val_adj)

## combining into a organized list and save
path = "./Results/CCC/"

multinichenet_output = list(
    celltype_info = abundance_expression_info$celltype_info,
    celltype_de = celltype_de,
    sender_receiver_info = abundance_expression_info$sender_receiver_info,
    sender_receiver_de =  sender_receiver_de,
    ligand_activities_targets_DEgenes = ligand_activities_targets_DEgenes,
    prioritization_tables = prioritization_tables,
    grouping_tbl = grouping_tbl,
    lr_target_prior_cor = lr_target_prior_cor
  ) 
multinichenet_output = make_lite_output(multinichenet_output)

save = TRUE
if(save == TRUE){
  saveRDS(multinichenet_output, paste0(path, "EC_multinichenet_output_0.25.rds"))

}

In [ ]:
## Astrocyte
prioritized_tbl_oi_all_1 = get_top_n_lr_pairs(multinichenet_output$prioritization_tables, 25, rank_per_group = F,
senders_oi = "Astrocyte",receivers_oi = c("Excitatory","Inhibitory","Microglia")) #,"Oligodendrocyte","OPC"
prioritized_tbl_oi_all_2 = get_top_n_lr_pairs(multinichenet_output$prioritization_tables, 25, rank_per_group = F,
receivers_oi = "Astrocyte",senders_oi = c("Excitatory","Inhibitory","Microglia")) #,"Oligodendrocyte","OPC"
prioritized_tbl_oi_all <- rbind(prioritized_tbl_oi_all_1,prioritized_tbl_oi_all_2)

## Figure 5a
prioritized_tbl_oi = multinichenet_output$prioritization_tables$group_prioritization_tbl %>%
  filter(id %in% prioritized_tbl_oi_all$id) %>%
  distinct(id, sender, receiver, ligand, receptor, group) %>% left_join(prioritized_tbl_oi_all)

prioritized_tbl_oi$prioritization_score[is.na(prioritized_tbl_oi$prioritization_score)] = 0

senders_receivers = union(prioritized_tbl_oi$sender %>% unique(), prioritized_tbl_oi$receiver %>% unique()) %>% sort()

colors_sender = c("#f06719ff","#33a65cff","#23767cff","#e03426ff","#1ba3c6ff","#a26dc2ff")%>% magrittr::set_names(senders_receivers)
colors_receiver = c("#f06719ff","#33a65cff","#23767cff","#e03426ff","#1ba3c6ff","#a26dc2ff")%>% magrittr::set_names(senders_receivers)

circos_list = make_circos_group_comparison(prioritized_tbl_oi, colors_sender, colors_receiver)

pdf("./Figures/CCC/EC_circo_ast_0.25.pdf",width = 7,height = 7)
circos_list
dev.off()

In [ ]:
## Microglia
prioritized_tbl_oi_all_1 = get_top_n_lr_pairs(multinichenet_output$prioritization_tables, 25, rank_per_group = F,
senders_oi = "Microglia",receivers_oi = c("Excitatory","Inhibitory","Astrocyte")) #,"Oligodendrocyte","OPC"
prioritized_tbl_oi_all_2 = get_top_n_lr_pairs(multinichenet_output$prioritization_tables, 25, rank_per_group = F,
receivers_oi = "Microglia",senders_oi = c("Excitatory","Inhibitory","Astrocyte")) #,"Oligodendrocyte","OPC"
prioritized_tbl_oi_all <- rbind(prioritized_tbl_oi_all_1,prioritized_tbl_oi_all_2)

## Figure 5b
prioritized_tbl_oi = multinichenet_output$prioritization_tables$group_prioritization_tbl %>%
  filter(id %in% prioritized_tbl_oi_all$id) %>%
  distinct(id, sender, receiver, ligand, receptor, group) %>% left_join(prioritized_tbl_oi_all)

prioritized_tbl_oi$prioritization_score[is.na(prioritized_tbl_oi$prioritization_score)] = 0

senders_receivers = union(prioritized_tbl_oi$sender %>% unique(), prioritized_tbl_oi$receiver %>% unique()) %>% sort()

colors_sender = c("#f06719ff","#33a65cff","#23767cff","#e03426ff","#1ba3c6ff","#a26dc2ff")%>% magrittr::set_names(senders_receivers)
colors_receiver = c("#f06719ff","#33a65cff","#23767cff","#e03426ff","#1ba3c6ff","#a26dc2ff")%>% magrittr::set_names(senders_receivers)

circos_list = make_circos_group_comparison(prioritized_tbl_oi, colors_sender, colors_receiver)

pdf("./Figures/CCC/EC_circo_mic_0.25.pdf",width = 7,height = 7)
circos_list
dev.off()

In [ ]:
### Figure 5c
group_oi = "NCI"
receiver_oi = "Excitatory"
sender_oi = "Astrocyte"
prioritized_tbl_oi_M_10 = get_top_n_lr_pairs(multinichenet_output$prioritization_tables, 5, groups_oi = group_oi, receivers_oi = receiver_oi,senders_oi = sender_oi)
# set case anc control
contrasts_oi = c("'EOAD-NCI','NCI-EOAD'")
contrast_tbl = tibble(contrast =
                        c("EOAD-NCI", "NCI-EOAD"),
                      group = c("EOAD", "NCI"))

combined_plot = make_ligand_activity_target_plot(
    group_oi, 
    receiver_oi, 
    prioritized_tbl_oi_M_10, 
    multinichenet_output$prioritization_tables, 
    multinichenet_output$ligand_activities_targets_DEgenes,
    contrast_tbl, 
    multinichenet_output$grouping_tbl, 
    multinichenet_output$celltype_info, 
    ligand_target_matrix, 
    plot_legend = FALSE)

pdf("./Figures/CCC/EC_Ast_Ex.pdf",width  = 13,height = 5)
combined_plot
dev.off()

In [ ]:
### Figure 5d
group_oi = "NCI"
receiver_oi = "Astrocyte"
sender_oi = "Microglia"
prioritized_tbl_oi_M_10 = get_top_n_lr_pairs(multinichenet_output$prioritization_tables, 4, groups_oi = group_oi, receivers_oi = receiver_oi,senders_oi = sender_oi)
# set case anc control
contrasts_oi = c("'EOAD-NCI','NCI-EOAD'")
contrast_tbl = tibble(contrast =
                        c("EOAD-NCI", "NCI-EOAD"),
                      group = c("EOAD", "NCI"))

combined_plot = make_ligand_activity_target_plot(
    group_oi, 
    receiver_oi, 
    prioritized_tbl_oi_M_10, 
    multinichenet_output$prioritization_tables, 
    multinichenet_output$ligand_activities_targets_DEgenes,
    contrast_tbl, 
    multinichenet_output$grouping_tbl, 
    multinichenet_output$celltype_info, 
    ligand_target_matrix, 
    plot_legend = FALSE)

pdf("./Figures/CCC/EC_Mic_Ast.pdf",width  = 13,height = 5)
combined_plot
dev.off()